# Classify scooter images



### Initialize and load model

In [1]:
# Set location of test data and saved models on google drive
data_path = "041320"
#data_path = "040520"
#image_dir = 'drive/My Drive/CoLab_Data/Hollywood/'
image_dir = 'drive/My Drive/CoLab_Data/LosAngeles/'
saved_model_dir = 'drive/My Drive/CoLab_Data/save/'


# import dependencies
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
%tensorflow_version 2.x
import tensorflow as tf
from google.colab import files
from google.colab import drive
from sklearn.metrics import classification_report, confusion_matrix

# connect to Google drive
drive.mount('/content/drive')

# unzip training images to local colab drive
os.system('rm %s%s' % (image_dir, data_path))
unzip_path = '%s%s.zip' % (image_dir, data_path)
print(unzip_path)
os.system('unzip "%s%s.zip"' % (image_dir, data_path))
os.listdir(data_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
drive/My Drive/CoLab_Data/LosAngeles/041320.zip


['1', '3', '0', '2']

In [2]:
# Load model directly 
model_name = 'mobileV2_LA_0407'
model = tf.keras.models.load_model(saved_model_dir + model_name + '.h5')

In [0]:
# or load different checkpoint for model
checkpoint = 'cp-0015.ckpt'
checkpoint_path = 'drive/My Drive/CoLab_Data/save/mobileV2_LA_0407'
model.load_weights(checkpoint_path + '/' + checkpoint)

In [4]:
# run inference on images

# define image size and classes
IMAGE_SIZE = 224
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
CLASS_NAMES=('bike_lane', 'crosswalk', 'side','street')

BATCH_SIZE=1000

# loop through folders
out_fnames=[]
for eval_folder in os.listdir(data_path):
  data_path_eval = data_path + '/' + eval_folder
  print(data_path_eval)

  # open data file for inference results
  outFile = image_dir + data_path + '_' + eval_folder + '_out_data.txt'
  out_fnames.append(outFile)
  outf = {'frame' : 0, 'label' : 1, 'class' : 2, 'swFlag' : 8, 'pos' : 9, 'vel' : 12}
  outfile_format = '%5s,%d,' + 6 * '%3.2f,' + '%d,' + 3 * '%14.9f,' + 3 * '%5.2f,' + '\n'
  fid = open(outFile, "w")

  # create image generator
  datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
  gen = datagen.flow_from_directory(
    data_path,
    classes=[eval_folder], 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle = False)
  num_images = gen.n

  # intialize loop variables
  ix=0
  y_pred = []
  # loop through images, one batch at a time
  for batch in range(num_images // BATCH_SIZE + 1):
    print(batch * BATCH_SIZE)
    # get next batch of images
    image_batch, label_batch = next(gen)


    # get predictions for this batch
    Y_pred = model.predict(image_batch)

    # add to complete set of predictions
    y_pred = np.concatenate((y_pred,np.argmax(Y_pred, axis=1)))

    # loop through images in batch
    for n in range(len(label_batch)):
      fn = gen.filenames[n+ix]
      img_num = fn[-19:-13] + fn[-12:-10] + fn[-9:-4]
      fid.write(outfile_format % (img_num, y_pred[ix+n], Y_pred[n][0], Y_pred[n][1],
              Y_pred[n][2], Y_pred[n][3], 0, 0, 0, 0, 0, 0, 0, 0, 0))
    ix += BATCH_SIZE

  fid.close()

041320/1
Found 2573 images belonging to 1 classes.
0
1000
2000
041320/3
Found 2600 images belonging to 1 classes.
0
1000
2000
041320/0
Found 680 images belonging to 1 classes.
0
041320/2
Found 3075 images belonging to 1 classes.
0
1000
2000
3000


In [0]:
# download results
for outFile in out_fnames:
  files.download(outFile)

In [0]:
#list type of GPU used

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Apr 17 18:49:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    33W / 250W |    385MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  